In [35]:
from langchain.document_loaders import PyPDFLoader

loaders = PyPDFLoader("constitution.pdf")
pages = loaders.load()

In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)

In [41]:
splits = text_splitter.split_documents(pages)

In [40]:
len(splits)

776

In [45]:
persist_directory = "DB/chroma"

In [46]:
from langchain_community.embeddings import GooglePalmEmbeddings
from langchain_google_genai import GoogleGenerativeAI
import os
from dotenv import load_dotenv

load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")

llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=google_api_key, temperature=0.7)

embeddings = GooglePalmEmbeddings(google_api_key=google_api_key)


In [47]:
from langchain_chroma import Chroma

vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory=persist_directory
)
vectordb.persist()


I0000 00:00:1725302444.625064 4185148 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


AttributeError: 'Chroma' object has no attribute 'persist'

In [48]:
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)


In [49]:
question = "Formation of new States and alteration of areas, boundaries or names of existing States"

In [50]:
docs = vectordb.similarity_search(question, k=2)

In [51]:
len(docs)

2

In [52]:
print(docs[0].page_content)

4. Subs. by the Uttaranchal (Alteration of Name) Act, 2006 (52 of 2006), s. 4, for the 
word "Uttaranchal" (w.e.f. 1-1-2007).
5. Added by the Bihar Reorganisation Act, 2000 (30 of 2000), s. 5 (w.e.f. 15-11-
2000).
6. Ins. by the Andhra Pradesh Reorganisation Act, 2014, s. 10 (w.e.f. 2-6-2014). 
7. Entry 2 relating to "Himachal Pradesh" omitted and entries 3 to 10 renumbered as 
entries 2 to 9 respectively by the State of Himachal Pradesh Act, 1970 (53 of 1970), 
s. 4 (w.e.f. 25-1-1971)and subsequently entries relating to Manipur and Tripura (i.e. 
entries 2 and 3) omitted by the North-Eastern Areas (Reorganisation) Act, 1971 (81 
of 1971) s. 9 (w.e.f. 21-1-1972).
8. Entries 4 to 9 renumbered as entries 2 to 7 by the North-Eastern Areas 
(Reorganisation) Act, 1971 (81 of 1971), s. 9 (w.e.f. 21-1-1972).


In [53]:
# vectordb.persist()

In [54]:
# source
docs = vectordb.similarity_search(question, k=3)
for doc in docs:
    print(doc.metadata)

{'page': 290, 'source': 'constitution.pdf'}
{'page': 288, 'source': 'constitution.pdf'}
{'page': 304, 'source': 'constitution.pdf'}


In [55]:
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [56]:
question = "Formation of new States and alteration of areas, boundaries or names of existing States"
docs = vectordb.similarity_search(question, k=2)
len(docs)

2

In [57]:
from langchain.chains import RetrievalQAWithSourcesChain

chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectordb.as_retriever())
chain

RetrievalQAWithSourcesChain(combine_documents_chain=MapReduceDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=GoogleGenerativeAI(model='models/text-bison-001', google_api_key=SecretStr('**********'), client=<module 'google.generativeai' from '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/google/generativeai/__init__.py'>)), reduce_documents_chain=ReduceDocumentsChain(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'summaries'], template='Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf you don\'t know the answer, just say that you don\'t know. Don\'t

In [58]:
# result = chain({"question": question})
# result

In [59]:
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say you dont know the answer. Dont make up the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [60]:
from langchain.chains.retrieval_qa.base import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
                                       return_source_documents=True,
                                       retriever=vectordb.as_retriever())

In [61]:
question = "Rights of citizenship of certain migrants to Pakistan"

In [62]:
result = qa_chain({"query": question})

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


In [63]:
result

{'query': 'Rights of citizenship of certain migrants to Pakistan',
 'result': 'Notwithstanding anything in articles 5 and 6, a person who has after the first day of March, 1947, migrated from the territory of India to the territory now included in Pakistan shall not be deemed to be a citizen of India:\n\nProvided that nothing in this article shall apply to a person who, after having so migrated to the territory now included in Pakistan, has returned to the territory of India under a permit for resettlement or permanent return issued by or under the authority of any law and every such person shall for the purposes of clause ( b) of article 6 be deemed to have migrated to the territory of India after the nineteenth day of July, 1948.',
 'source_documents': [Document(metadata={'page': 34, 'source': 'constitution.pdf'}, page_content='Dominion of India on an application made by him therefor to such \nofficer before the commencement of this Constitution in the form and \nmanner prescribed by

In [64]:
print(result["result"])

Notwithstanding anything in articles 5 and 6, a person who has after the first day of March, 1947, migrated from the territory of India to the territory now included in Pakistan shall not be deemed to be a citizen of India:

Provided that nothing in this article shall apply to a person who, after having so migrated to the territory now included in Pakistan, has returned to the territory of India under a permit for resettlement or permanent return issued by or under the authority of any law and every such person shall for the purposes of clause ( b) of article 6 be deemed to have migrated to the territory of India after the nineteenth day of July, 1948.


In [65]:
print(result["source_documents"][0])

page_content='Dominion of India on an application made by him therefor to such 
officer before the commencement of this Constitution in the form and 
manner prescribed by that Government:
Provided that no person shall be so registered unless he has been resident 
in the territory of India for at least six months immediately preceding the date 
of his application.
7. Rights of citizenship of certain migrants to Pakistan .—
Notwithstanding anything in articles 5 and 6, a person who has after the first 
day of March, 1947, migrated from the territory of India to the territory now 
included in Pakistan shall not be deemed to be a citizen of India:' metadata={'page': 34, 'source': 'constitution.pdf'}


In [66]:
#map reduce

qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type="map_reduce",
                                       retriever=vectordb.as_retriever())

In [67]:
question = "rights of india"
result = qa_chain({"query": question})

In [68]:
print(result["result"])

to reside and settle in any part of the territory of India; to practise any profession, or to carry on any occupation, trade or business


In [69]:
#build prompt

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say you dont know the answer. Dont make up the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"], template=template)


In [70]:
#run chain
question = "What are the rights of woman in india?"

qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
                                       return_source_documents=True,
                                       retriever=vectordb.as_retriever())
result = qa_chain({"query": question})
print(result['result'])

Article 15(3) provides that Nothing in this article shall prevent the State from making any special provision for women and children.


In [71]:
#chat history buffer
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [72]:
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain

retriever = vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)


In [73]:
question = "What does it say specifically say about equality?"
result = qa({"question": question})

In [74]:
print(result["answer"])

Article 14 states that all persons are equal before the law and article 15 prohibits discrimination on grounds of religion, race, caste, sex, place of birth or any of them.


In [75]:
question = "what was the constitution before 1930?"
result = qa({"question": question})
print(result["answer"])

The Government of India Act 1935 was the constitution of India before 1930.


In [76]:
question = "what all are the fundamental rights of a person"
result = qa({"question": question})
print(result["answer"])

- Right to equality
- Right to freedom of speech and expression
- Right to assembly
- Right to form associations
- Right to move freely throughout the territory of India
- Right to reside and settle in any part of the territory of India
- Right to practise any profession, or to carry on any occupation, trade or business
